In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
def extract_noun_phrases(text):
    return [chunk.text for chunk in nlp(text).noun_chunks]

def extract_verb_phrases(text):
    return [chunk.text for chunk in nlp(text).verb]

In [3]:
def extract_compounds(text):
    comp_idx = 0
    compound = []
    compound_nps = []
    tok_idx = 0
    for idx, tok in enumerate(nlp(text)):
        if tok.dep_ == 'compound':
            children = ''.join([c.text for c in tok.children])
            if '-' in children:
                compound.append(''.join([children, tok.text]))
            else:
                compound.append(tok.text)
            try:
                tok_idx = [c for c in tok.children][0].idx
            except IndexError:
                if len(compound) == 1:
                    tok_idx = tok.idx
            comp_idx = tok.i
        if tok.i - comp_idx == 1:
            compound.append(tok.text)
            if len(compound) > 1:
                compound = ' '.join(compound)
                compound_nps.append(compound)
            tok_idx = 0
            compound = []
    return list(set(compound_nps))

In [4]:
def extract_named_ents(text):
    return [ent.text for ent in nlp(text).ents]

In [5]:
from spacy import displacy

In [6]:
sentence = 'the american civil liberties union staged a nationwide training event saturday to make sure people are aware of their rights as protesters and urge organized, public resistance by those opposed to policies of president donald trump.'

doc = nlp(sentence)

print ("{:<15} | {:<8} | {:<15} | {:<20}".format('Token','Relation','Head', 'Children'))
print ("-" * 70)

for token in doc:
  print ("{:<15} | {:<8} | {:<15} | {:<20}"
         .format(str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))

displacy.render(doc, style='dep', jupyter=True, options={'distance': 120})

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
the             | det      | union           | []                  
american        | amod     | union           | []                  
civil           | amod     | liberties       | []                  
liberties       | compound | union           | [civil]             
union           | nsubj    | staged          | [the, american, liberties]
staged          | ROOT     | staged          | [union, event, saturday, make, resistance, .]
a               | det      | event           | []                  
nationwide      | amod     | event           | []                  
training        | compound | event           | []                  
event           | dobj     | staged          | [a, nationwide, training]
saturday        | npadvmod | staged          | []                  
to              | aux      | make            | []                  
make     